In [1]:
import torch
import numpy as np

In [2]:
# FizzBuzz游戏：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数
# 只有四种可能，将其编码
def fb_encode(i):
    if i % 15 == 0:
        return 3
    elif i % 5 == 0:
        return 2
    elif i % 3 == 0:
        return 1
    else:
        return 0
    
def fb_decode(i,x):
    return [str(i),'fizz','buzz','fizzbuzz'][x]
out = []
for i in range(1,16):
    out.append(fb_decode(i,fb_encode(i)))
print(out)

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz']


In [3]:
#我们只考虑2 ** 10以内的数据

NUM_DIGITS = 10

#将其转成2进制的形式输入，input dimention是10维的
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

#随机101-end作为训练的输入
trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fb_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

In [4]:
hidden = 100
model = torch.nn.Sequential(
    torch.nn.Linear(10,hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden,4)
)

# use_cuda = torch.cuda.is_available()
# if use_cuda:
#     model = model.cuda()
    
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.05)

In [5]:
# epoch = 1e4
batch_size = 128

# if use_cuda:
#     model = model.cuda()
#     trX = trX.cuda()
#     trY = trY.cuda()

for epoch in range(10000):
    for start in range(0,len(trX),batch_size):
        end = start + batch_size
        batch_X = trX[start:end]
        batch_Y = trY[start:end]
        
        y_pred = model(batch_X)
        loss = loss_fn(y_pred,batch_Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 2000 ==0:     
        loss = loss_fn(model(trX),trY).item()
        print("epoch:{0} {1}".format(epoch,loss))

epoch:0 1.2046138048171997
epoch:2000 0.11422817409038544
epoch:4000 0.035096824169158936
epoch:6000 0.016180429607629776
epoch:8000 0.009747601114213467


In [6]:
#1-100留作测试
testX = torch.Tensor([binary_encode(i,10) for i in range(1,101)])
# model_out = model.cpu()
with torch.no_grad():
    testY = model(testX)
predic = zip(range(1,101),list(testY.max(1)[1].data.tolist()))

In [7]:
res = []
for i,x in predic:
    res.append(fb_decode(i,x))
print(res)

['1', '2', 'fizz', 'fizz', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', '21', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', '69', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', '81', '82', '83', 'fizz', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [9]:
print(np.sum(testY.max(1)[1].numpy()==np.array([fb_encode(i) for i in range(1,101)])))

95
